In [9]:
sc = SparkContext.getOrCreate()

In [10]:
data = sc.parallelize(
[('Amber',22),('Alfred',23),('Skye',4),('Albert',12),('Amber',9)])
print(data)
print(data.collect())

ParallelCollectionRDD[24] at readRDDFromFile at PythonRDD.scala:262
[('Amber', 22), ('Alfred', 23), ('Skye', 4), ('Albert', 12), ('Amber', 9)]


In [11]:
import numpy as np
rData = [np.random.randn() for _ in range(10)]
print(rData)

[0.3170665842030632, 1.2731134390663263, -0.5389280846540201, 0.024542946433471123, 1.0897156478413264, -0.5437300629230136, 0.9126056955713316, 0.25952856976275246, -1.9911240821785927, 0.882740335467687]


In [12]:
data = sc.parallelize(rData)
print("data\n", data) #해당 context의 정보 표기
print("all data\n", data.collect())
print("the first 5\n", data.take(5))

data
 ParallelCollectionRDD[25] at readRDDFromFile at PythonRDD.scala:262
all data
 [0.3170665842030632, 1.2731134390663263, -0.5389280846540201, 0.024542946433471123, 1.0897156478413264, -0.5437300629230136, 0.9126056955713316, 0.25952856976275246, -1.9911240821785927, 0.882740335467687]
the first 5
 [0.3170665842030632, 1.2731134390663263, -0.5389280846540201, 0.024542946433471123, 1.0897156478413264]


In [13]:
data = sc.parallelize([("park",43),("kim",25)])
print(data.collect())

[('park', 43), ('kim', 25)]


In [14]:
data = sc.textFile("./test1.txt")
data.take(10)

['ROMEO AND JULIET',
 '',
 '',
 'ACT I',
 '',
 '',
 '',
 'SCENE I\tVerona. A public place.',
 '',
 '']

In [15]:
# map 
x = sc.parallelize(["b","a","c"])
print(x.collect())
print(x)

['b', 'a', 'c']
ParallelCollectionRDD[32] at readRDDFromFile at PythonRDD.scala:262


In [16]:
y = x.map(lambda z : (z,1))
print(y.collect())
print(y)

[('b', 1), ('a', 1), ('c', 1)]
PythonRDD[33] at collect at <ipython-input-16-cfaae7e231aa>:2


In [17]:
# filter
x = sc.parallelize([1,2,3])
y = x.filter(lambda x : x%2 == 1) # 홀수만 골라내기
print(x.collect())
print(y.collect())

[1, 2, 3]
[1, 3]


In [18]:
# flatmap
x = sc.parallelize([1,2,3])
y = x.flatMap(lambda x : (x, x * 100, 42))
print(x.collect())
print(y.collect())

[1, 2, 3]
[1, 100, 42, 2, 200, 42, 3, 300, 42]


In [19]:
# map과 비교
x = sc.parallelize([1,2,3])
y = x.map(lambda x : (x, x * 100, 42))
print(x.collect())
print(y.collect())

[1, 2, 3]
[(1, 100, 42), (2, 200, 42), (3, 300, 42)]


In [20]:
# groupBy에서는 함수 또는 키를 지정해야함, groupByKey라는 것도 있는데 이거는 key를 이미 알고 있을때
x = sc.parallelize(['John', 'Fred', 'Anna', 'James'])
y = x.groupBy(lambda w : w[0]) # key 값을 결정해주면, key값에 따라 그룹이 나뉨
print([(k, list(v)) for (k,v) in y.collect()])

[('J', ['John', 'James']), ('F', ['Fred']), ('A', ['Anna'])]


In [21]:
print(y)

PythonRDD[45] at collect at <ipython-input-20-62372437a948>:4


In [22]:
[(k,v) for k,v in y.collect()] # list형태로 지정해서 출력해야한다

[('J', <pyspark.resultiterable.ResultIterable at 0x199ae06e100>),
 ('F', <pyspark.resultiterable.ResultIterable at 0x199ae06e1f0>),
 ('A', <pyspark.resultiterable.ResultIterable at 0x199adfc9220>)]

In [23]:
# groupBykey
x = sc.parallelize([('B',5),('B',4),('A',3),('A',2),('A',1)])
y = x.groupByKey()
print(x.collect())
print(list((k, list(v)) for (k,v) in y.collect()))

[('B', 5), ('B', 4), ('A', 3), ('A', 2), ('A', 1)]
[('B', [5, 4]), ('A', [3, 2, 1])]


In [24]:
x = sc.parallelize([('B',5),('B',4),('A',3),('A',2),('A',1)])
y = x.groupByKey()
print(x.collect())
print(list((j[0], list(j[1])) for j in y.collect()))

[('B', 5), ('B', 4), ('A', 3), ('A', 2), ('A', 1)]
[('B', [5, 4]), ('A', [3, 2, 1])]


In [25]:
y.collect()

[('B', <pyspark.resultiterable.ResultIterable at 0x199adfc95b0>),
 ('A', <pyspark.resultiterable.ResultIterable at 0x199adfcd160>)]

In [26]:
# reduceBYKey vs groupByKey, Key가 있을경우 reduceBYKey가 더 효율적임
words = ["one", "two", "two", "three", "three", "three"]
wordPairsRDD = sc.parallelize(words).map(lambda x : (x,1))
wordPairsRDD.collect()

[('one', 1), ('two', 1), ('two', 1), ('three', 1), ('three', 1), ('three', 1)]

In [27]:
wordsCountsWithReduce = wordPairsRDD.reduceByKey(lambda x,y : x+y).collect() #미리 value들을 count하고나서 컴퓨터에 배치
wordsCountsWithReduce

[('two', 2), ('three', 3), ('one', 1)]

In [28]:
wordCountsWithGroup = wordPairsRDD.groupByKey().map(lambda x : (x[0], sum(x[1]))).collect() # value들을 먼저 컴퓨터에 배치하고 나서 나중에 모두 count
wordCountsWithGroup

[('two', 2), ('three', 3), ('one', 1)]

In [29]:
# Partition and mapPartition
data = sc.parallelize(np.arange(20))
print(data.glom().collect())
data1 = sc.parallelize(np.arange(20),4)
print(data1.glom().collect())

[[0, 1], [2, 3], [4, 5], [6, 7, 8, 9], [10, 11], [12, 13], [14, 15], [16, 17, 18, 19]]
[[0, 1, 2, 3, 4], [5, 6, 7, 8, 9], [10, 11, 12, 13, 14], [15, 16, 17, 18, 19]]


In [30]:
print(data.collect())
print(data1.collect())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [32]:
x = sc.parallelize(np.arange(10),2)
def f(iterator) :
    yield sum(iterator)
y = x.mapPartitions(f)
# glom() flattens elements on the same parition, 즉 파티션 내의 요소들을 쭉 리스트로 나열한다
print(x.glom().collect())
print(y.glom().collect())

[[0, 1, 2, 3, 4], [5, 6, 7, 8, 9]]
[[10], [35]]


In [33]:
print(x.collect())
print(y.collect())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[10, 35]


In [36]:
# mapPartitionsWithIndex
x = sc.parallelize(np.arange(10),2)
def f(partitionIndex, iterator) :
    yield(partitionIndex, sum(iterator))
y = x.mapPartitionsWithIndex(f)

print(x.glom().collect())
print(y.glom().collect())

[[0, 1, 2, 3, 4], [5, 6, 7, 8, 9]]
[[(0, 10)], [(1, 35)]]
